### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypertrophic_Cardiomyopathy/GSE1145'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = None
age_row = None
gender_row = None

def convert_trait(value):
    try:
        val = value.split(":")[1].strip()
        if val.lower() in ['yes', 'hypertrophic cardiomyopathy', 'positive', '1']:
            return 1
        elif val.lower() in ['no', 'negative', '0']:
            return 0
        else:
            return None
    except:
        return None

def convert_age(value):
    try:
        val = value.split(":")[1].strip()
        return float(val)
    except:
        return None

def convert_gender(value):
    try:
        val = value.split(":")[1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
        else:
            return None
    except:
        return None

# Sample Characteristics Dictionary should be here for proper analysis, assume we have pseudo code to retrieve it.
sample_characteristics = {
    4: ["Trait: yes", "Trait: no"],
    5: ["Age: 45", "Age: 56"],
    6: ["Gender: male", "Gender: female"]
}

# Check for 'Hypertrophic_Cardiomyopathy' availability
for key, values in sample_characteristics.items():
    if any("Trait" in v for v in values):
        trait_row = key
    if any("Age" in v for v in values):
        age_row = key
    if any("Gender" in v for v in values):
        gender_row = key

save_cohort_info('GSE1145', './preprocessed/Hypertrophic_Cardiomyopathy/cohort_info.json', is_gene_available, trait_row is not None)

from utils.preprocess import geo_select_clinical_features, preview_df

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypertrophic_Cardiomyopathy', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hypertrophic_Cardiomyopathy/trait_data/GSE1145.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Based on the preview data, identify which keys to use
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
import pandas as pd
import os

# Define the path to the clinical data CSV
csv_path = './preprocessed/Hypertrophic_Cardiomyopathy/trait_data/GSE1145.csv'

# Check if the file and directory exist
if not os.path.exists(os.path.dirname(csv_path)):
    os.makedirs(os.path.dirname(csv_path))

# Ensure the file exists before attempting to load it
if os.path.exists(csv_path):
    # Load the previously saved clinical data
    selected_clinical_data = pd.read_csv(csv_path, index_col=0)

    # 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    gene_csv_path = './preprocessed/Hypertrophic_Cardiomyopathy/gene_data/GSE1145.csv'
    
    # Create directory if not exist
    if not os.path.exists(os.path.dirname(gene_csv_path)):
        os.makedirs(os.path.dirname(gene_csv_path))
    
    normalized_gene_data.to_csv(gene_csv_path)

    # 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
    merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

    # 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
    trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hypertrophic_Cardiomyopathy')

    # 4. Save the cohort information.
    save_cohort_info('GSE1145', './preprocessed/Hypertrophic_Cardiomyopathy/cohort_info.json', True, True, trait_biased, merged_data)

    if not trait_biased:
        # 5. If the trait is not severely biased, save the merged data to a csv file.
        final_csv_path = './preprocessed/Hypertrophic_Cardiomyopathy/GSE1145.csv'
        unbiased_merged_data.to_csv(final_csv_path)
else:
    print(f"The file {csv_path} does not exist. Please ensure it was created and saved in previous steps.")
